# **Car Crashes' Severity Prediction**

# > **IMPORT LIBERIRES AND READ THE DATA**

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
dataset_path = '/kaggle/input/car-crashes-severity-prediction/'

df = pd.read_csv(os.path.join(dataset_path, 'train.csv'))

print("The shape of the dataset is {}.\n\n".format(df.shape))

df.head()

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

The shape of the dataset is (6407, 16).




,ID,Lat,Lng,Bump,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Stop,Amenity,Side,Severity,timestamp
0,0,37.762150,-122.405660,False,0.044,False,False,False,False,False,False,False,True,R,2,2016-03-25 15:13:02
1,1,37.719157,-122.448254,False,0.000,False,False,False,False,False,False,False,False,R,2,2020-05-05 19:23:00
2,2,37.808498,-122.366852,False,0.000,False,False,False,False,False,False,True,False,R,3,2016-09-16 19:57:16
3,3,37.785930,-122.391080,False,0.009,False,False,True,False,False,False,False,False,R,1,2020-03-29 19:48:43
4,4,37.719141,-122.448457,False,0.000,False,False,False,False,False,False,False,False,R,2,2019-10-09 08:47:00


# > **EDA**

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6407 entries, 0 to 6406
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ID            6407 non-null   int64  
 1   Lat           6407 non-null   float64
 2   Lng           6407 non-null   float64
 3   Bump          6407 non-null   bool   
 4   Distance(mi)  6407 non-null   float64
 5   Crossing      6407 non-null   bool   
 6   Give_Way      6407 non-null   bool   
 7   Junction      6407 non-null   bool   
 8   No_Exit       6407 non-null   bool   
 9   Railway       6407 non-null   bool   
 10  Roundabout    6407 non-null   bool   
 11  Stop          6407 non-null   bool   
 12  Amenity       6407 non-null   bool   
 13  Side          6407 non-null   object 
 14  Severity      6407 non-null   int64  
 15  timestamp     6407 non-null   object 
dtypes: bool(9), float64(3), int64(2), object(2)
memory usage: 406.8+ KB


In [3]:
df.describe()

,ID,Lat,Lng,Distance(mi),Severity
count,6407.000000,6407.000000,6407.000000,6407.000000,6407.000000
mean,3203.000000,37.765653,-122.405990,0.135189,2.293429
std,1849.685919,0.032555,0.028275,0.396360,0.521225
min,0.000000,37.609619,-122.510440,0.000000,1.000000
25%,1601.500000,37.737096,-122.412210,0.000000,2.000000
50%,3203.000000,37.768238,-122.404835,0.000000,2.000000
75%,4804.500000,37.787813,-122.392477,0.041000,3.000000
max,6406.000000,37.825626,-122.349734,6.820000,4.000000


In [4]:
df.isnull().sum()

ID              0
Lat             0
Lng             0
Bump            0
Distance(mi)    0
Crossing        0
Give_Way        0
Junction        0
No_Exit         0
Railway         0
Roundabout      0
Stop            0
Amenity         0
Side            0
Severity        0
timestamp       0
dtype: int64

# > **PREPROCESSING**

In [5]:
df["Bump"] = df["Bump"].astype(int)
df["Crossing"] = df["Crossing"].astype(int)
df["Give_Way"] = df["Give_Way"].astype(int)
df["Junction"] = df["Junction"].astype(int)
df["No_Exit"] = df["No_Exit"].astype(int)
df["Railway"] = df["Railway"].astype(int)
df["Roundabout"] = df["Roundabout"].astype(int)
df["Stop"] = df["Stop"].astype(int)
df["Amenity"] = df["Amenity"].astype(int)
df.head()

,ID,Lat,Lng,Bump,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Stop,Amenity,Side,Severity,timestamp
0,0,37.762150,-122.405660,0,0.044,0,0,0,0,0,0,0,1,R,2,2016-03-25 15:13:02
1,1,37.719157,-122.448254,0,0.000,0,0,0,0,0,0,0,0,R,2,2020-05-05 19:23:00
2,2,37.808498,-122.366852,0,0.000,0,0,0,0,0,0,1,0,R,3,2016-09-16 19:57:16
3,3,37.785930,-122.391080,0,0.009,0,0,1,0,0,0,0,0,R,1,2020-03-29 19:48:43
4,4,37.719141,-122.448457,0,0.000,0,0,0,0,0,0,0,0,R,2,2019-10-09 08:47:00


In [6]:
f=list(df["timestamp"])                                   #convtring time 
new_time=[]
for i in range (len(f)):
    new_time.append(f[i].split()[0])
df["Time"]=new_time



In [7]:
del df['Side']
del df['timestamp']
df

,ID,Lat,Lng,Bump,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Stop,Amenity,Severity,Time
0,0,37.762150,-122.405660,0,0.044,0,0,0,0,0,0,0,1,2,2016-03-25
1,1,37.719157,-122.448254,0,0.000,0,0,0,0,0,0,0,0,2,2020-05-05
2,2,37.808498,-122.366852,0,0.000,0,0,0,0,0,0,1,0,3,2016-09-16
3,3,37.785930,-122.391080,0,0.009,0,0,1,0,0,0,0,0,1,2020-03-29
4,4,37.719141,-122.448457,0,0.000,0,0,0,0,0,0,0,0,2,2019-10-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6402,6402,37.740630,-122.407930,0,0.368,0,0,0,0,0,0,0,0,3,2017-10-01
6403,6403,37.752755,-122.402790,0,0.639,0,0,1,0,0,0,0,0,2,2018-10-23
6404,6404,37.726304,-122.446015,0,0.000,0,0,1,0,0,0,0,0,2,2019-10-28
6405,6405,37.808090,-122.367211,0,0.000,0,0,1,0,0,0,0,0,3,2019-05-04


In [8]:
df.drop(columns='ID').describe()

,Lat,Lng,Bump,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Stop,Amenity,Severity
count,6407.000000,6407.000000,6407.0,6407.000000,6407.00000,6407.000000,6407.000000,6407.000000,6407.000000,6407.0,6407.000000,6407.000000,6407.000000
mean,37.765653,-122.405990,0.0,0.135189,0.08241,0.000468,0.246449,0.000156,0.026533,0.0,0.097706,0.037147,2.293429
std,0.032555,0.028275,0.0,0.396360,0.27501,0.021635,0.430977,0.012493,0.160728,0.0,0.296939,0.189136,0.521225
min,37.609619,-122.510440,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,1.000000
25%,37.737096,-122.412210,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,2.000000
50%,37.768238,-122.404835,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,2.000000
75%,37.787813,-122.392477,0.0,0.041000,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,3.000000
max,37.825626,-122.349734,0.0,6.820000,1.00000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,4.000000


In [9]:
dataset_path = '/kaggle/input/car-crashes-severity-prediction/'

df2 = pd.read_csv(os.path.join(dataset_path, 'weather-sfcsv.csv'))

print("The shape of the dataset is {}.\n\n".format(df.shape))


The shape of the dataset is (6407, 15).




In [10]:
time=[]
d=list(df2["Day"])
m=list(df2["Month"])
y=list(df2["Year"])
for i in range(len(y)):
    s=str(y[i])+"-"+str(m[i])+"-"+str(d[i])
    time.append(s)
del df2['Day']
del df2['Year']
del df2['Month']
del df2['Hour']

df2["Time"]=time
df2

,Weather_Condition,Wind_Chill(F),Precipitation(in),Temperature(F),Humidity(%),Wind_Speed(mph),Visibility(mi),Selected,Time
0,Fair,64.0,0.00,64.0,70.0,20.0,10.0,No,2020-7-27
1,Partly Cloudy,NaN,NaN,71.1,57.0,9.2,10.0,No,2017-9-30
2,Overcast,NaN,NaN,57.9,87.0,15.0,9.0,No,2017-6-27
3,Clear,NaN,NaN,66.9,73.0,4.6,10.0,No,2016-9-7
4,Fair,52.0,0.00,52.0,89.0,0.0,9.0,No,2019-10-19
...,...,...,...,...,...,...,...,...,...
6896,Clear,NaN,NaN,51.1,80.0,3.5,10.0,No,2018-1-23
6897,Cloudy,56.0,0.00,56.0,80.0,9.0,9.0,No,2019-6-16
6898,Rain,NaN,0.07,61.0,90.0,32.2,7.0,No,2017-2-7
6899,Mostly Cloudy,NaN,NaN,61.0,67.0,21.9,10.0,No,2016-4-22


In [11]:
df2=df2.drop_duplicates(['Time'], keep='last')
print("The shape of the Weather After removing dublicate is {}.\n\n".format(df2.shape))

The shape of the Weather After removing dublicate is (1592, 9).




In [12]:
del df2["Selected"]
df2

,Weather_Condition,Wind_Chill(F),Precipitation(in),Temperature(F),Humidity(%),Wind_Speed(mph),Visibility(mi),Time
0,Fair,64.0,0.00,64.0,70.0,20.0,10.0,2020-7-27
51,Fair,71.0,0.00,71.0,61.0,7.0,10.0,2019-6-23
63,Clear,NaN,NaN,59.0,22.0,9.2,10.0,2018-2-10
75,Scattered Clouds,NaN,NaN,66.9,73.0,5.8,10.0,2017-8-20
88,Fair,66.0,0.00,66.0,60.0,20.0,10.0,2019-5-8
...,...,...,...,...,...,...,...,...
6896,Clear,NaN,NaN,51.1,80.0,3.5,10.0,2018-1-23
6897,Cloudy,56.0,0.00,56.0,80.0,9.0,9.0,2019-6-16
6898,Rain,NaN,0.07,61.0,90.0,32.2,7.0,2017-2-7
6899,Mostly Cloudy,NaN,NaN,61.0,67.0,21.9,10.0,2016-4-22


In [13]:
import xml.etree.ElementTree as et 

xtree = et.parse(os.path.join(dataset_path, 'holidays.xml'))
xroot = xtree.getroot() 

df_cols = ["date", "description"]
rows = []

for node in xroot: 
    s_description = node.find("description").text 
    s_date = node.find("date").text 
    
    
    rows.append({"description": s_description, "date": s_date})

df3 = pd.DataFrame(rows, columns = df_cols)
df3

,date,description
0,2012-01-02,New Year Day
1,2012-01-16,Martin Luther King Jr. Day
2,2012-02-20,Presidents Day (Washingtons Birthday)
3,2012-05-28,Memorial Day
4,2012-07-04,Independence Day
...,...,...
85,2020-09-07,Labor Day
86,2020-10-12,Columbus Day
87,2020-11-11,Veterans Day
88,2020-11-26,Thanksgiving Day


In [14]:
df3.rename(columns = {'date':'Time'}, inplace = True)
df3

,Time,description
0,2012-01-02,New Year Day
1,2012-01-16,Martin Luther King Jr. Day
2,2012-02-20,Presidents Day (Washingtons Birthday)
3,2012-05-28,Memorial Day
4,2012-07-04,Independence Day
...,...,...
85,2020-09-07,Labor Day
86,2020-10-12,Columbus Day
87,2020-11-11,Veterans Day
88,2020-11-26,Thanksgiving Day


In [15]:
x=df.merge(df2,how="left",on=['Time'])
y=x.merge(df3,how="left",on=['Time'])
y

,ID,Lat,Lng,Bump,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,...,Severity,Time,Weather_Condition,Wind_Chill(F),Precipitation(in),Temperature(F),Humidity(%),Wind_Speed(mph),Visibility(mi),description
0,0,37.762150,-122.405660,0,0.044,0,0,0,0,0,...,2,2016-03-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,37.719157,-122.448254,0,0.000,0,0,0,0,0,...,2,2020-05-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,37.808498,-122.366852,0,0.000,0,0,0,0,0,...,3,2016-09-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,37.785930,-122.391080,0,0.009,0,0,1,0,0,...,1,2020-03-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,37.719141,-122.448457,0,0.000,0,0,0,0,0,...,2,2019-10-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6402,6402,37.740630,-122.407930,0,0.368,0,0,0,0,0,...,3,2017-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6403,6403,37.752755,-122.402790,0,0.639,0,0,1,0,0,...,2,2018-10-23,Mostly Cloudy,NaN,NaN,64.0,56.0,9.2,10.0,NaN
6404,6404,37.726304,-122.446015,0,0.000,0,0,1,0,0,...,2,2019-10-28,Fair,65.0,0.0,65.0,14.0,5.0,10.0,NaN
6405,6405,37.808090,-122.367211,0,0.000,0,0,1,0,0,...,3,2019-05-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
del y["Weather_Condition"]
del y["Give_Way"]
del y["description"]
del y["Crossing"]
y

,ID,Lat,Lng,Bump,Distance(mi),Junction,No_Exit,Railway,Roundabout,Stop,Amenity,Severity,Time,Wind_Chill(F),Precipitation(in),Temperature(F),Humidity(%),Wind_Speed(mph),Visibility(mi)
0,0,37.762150,-122.405660,0,0.044,0,0,0,0,0,1,2,2016-03-25,NaN,NaN,NaN,NaN,NaN,NaN
1,1,37.719157,-122.448254,0,0.000,0,0,0,0,0,0,2,2020-05-05,NaN,NaN,NaN,NaN,NaN,NaN
2,2,37.808498,-122.366852,0,0.000,0,0,0,0,1,0,3,2016-09-16,NaN,NaN,NaN,NaN,NaN,NaN
3,3,37.785930,-122.391080,0,0.009,1,0,0,0,0,0,1,2020-03-29,NaN,NaN,NaN,NaN,NaN,NaN
4,4,37.719141,-122.448457,0,0.000,0,0,0,0,0,0,2,2019-10-09,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6402,6402,37.740630,-122.407930,0,0.368,0,0,0,0,0,0,3,2017-10-01,NaN,NaN,NaN,NaN,NaN,NaN
6403,6403,37.752755,-122.402790,0,0.639,1,0,0,0,0,0,2,2018-10-23,NaN,NaN,64.0,56.0,9.2,10.0
6404,6404,37.726304,-122.446015,0,0.000,1,0,0,0,0,0,2,2019-10-28,65.0,0.0,65.0,14.0,5.0,10.0
6405,6405,37.808090,-122.367211,0,0.000,1,0,0,0,0,0,3,2019-05-04,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
y['Visibility(mi)'].fillna((y['Visibility(mi)'].mean()), inplace=True)
y['Precipitation(in)'].fillna((y['Precipitation(in)'].mean()), inplace=True)
y['Wind_Chill(F)'].fillna((y['Wind_Chill(F)'].mean()), inplace=True)
y['Temperature(F)'].fillna((y['Temperature(F)'].mean()), inplace=True)
y['Humidity(%)'].fillna((y['Humidity(%)'].mean()), inplace=True)
y['Wind_Speed(mph)'].fillna((y['Wind_Speed(mph)'].mean()), inplace=True)

In [18]:
y

,ID,Lat,Lng,Bump,Distance(mi),Junction,No_Exit,Railway,Roundabout,Stop,Amenity,Severity,Time,Wind_Chill(F),Precipitation(in),Temperature(F),Humidity(%),Wind_Speed(mph),Visibility(mi)
0,0,37.762150,-122.405660,0,0.044,0,0,0,0,0,1,2,2016-03-25,56.661145,0.005575,57.396212,67.367424,8.550732,9.167348
1,1,37.719157,-122.448254,0,0.000,0,0,0,0,0,0,2,2020-05-05,56.661145,0.005575,57.396212,67.367424,8.550732,9.167348
2,2,37.808498,-122.366852,0,0.000,0,0,0,0,1,0,3,2016-09-16,56.661145,0.005575,57.396212,67.367424,8.550732,9.167348
3,3,37.785930,-122.391080,0,0.009,1,0,0,0,0,0,1,2020-03-29,56.661145,0.005575,57.396212,67.367424,8.550732,9.167348
4,4,37.719141,-122.448457,0,0.000,0,0,0,0,0,0,2,2019-10-09,56.661145,0.005575,57.396212,67.367424,8.550732,9.167348
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6402,6402,37.740630,-122.407930,0,0.368,0,0,0,0,0,0,3,2017-10-01,56.661145,0.005575,57.396212,67.367424,8.550732,9.167348
6403,6403,37.752755,-122.402790,0,0.639,1,0,0,0,0,0,2,2018-10-23,56.661145,0.005575,64.000000,56.000000,9.200000,10.000000
6404,6404,37.726304,-122.446015,0,0.000,1,0,0,0,0,0,2,2019-10-28,65.000000,0.000000,65.000000,14.000000,5.000000,10.000000
6405,6405,37.808090,-122.367211,0,0.000,1,0,0,0,0,0,3,2019-05-04,56.661145,0.005575,57.396212,67.367424,8.550732,9.167348


# > **SPLIT DATA**

In [19]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(y, test_size=0.2, random_state=42) # Try adding `stratify` here

X_train = train_df.drop(columns=['ID', 'Severity'])
y_train = train_df['Severity']

X_val = val_df.drop(columns=['ID', 'Severity'])
y_val = val_df['Severity']

# > **FEATURES SELECTION FOR TRAINING**

In [20]:
# This cell is used to select the numerical features. IT SHOULD BE REMOVED AS YOU DO YOUR WORK.
X_train = X_train[['Lat', 'Lng', 'Distance(mi)','Junction','Stop']]
X_val = X_val    [['Lat', 'Lng', 'Distance(mi)','Junction','Stop']]

# > **MACHINE LEARNING MODELS**

In [21]:
from sklearn.ensemble import RandomForestClassifier

# Create an instance of the classifier
classifier = RandomForestClassifier(max_depth=2, random_state=0)

# Train the classifier
classifier = classifier.fit(X_train, y_train)

In [22]:
print("The accuracy of the classifier on the validation set is ", (classifier.score(X_val, y_val)))

The accuracy of the classifier on the validation set is  0.7449297971918877


In [23]:
feature_imp = pd.Series(classifier.feature_importances_,index=list(X_train.columns.values))
feature_imp

Lat             0.351833
Lng             0.353411
Distance(mi)    0.118850
Junction        0.012375
Stop            0.163532
dtype: float64

# > **READ TEST DATA**

In [24]:
test_df = pd.read_csv(os.path.join(dataset_path, 'test.csv'))
test_df.head()

,ID,Lat,Lng,Bump,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Stop,Amenity,Side,timestamp
0,6407,37.786060,-122.390900,False,0.039,False,False,True,False,False,False,False,False,R,2016-04-04 19:20:31
1,6408,37.769609,-122.415057,False,0.202,False,False,False,False,False,False,False,False,R,2020-10-28 11:51:00
2,6409,37.807495,-122.476021,False,0.000,False,False,False,False,False,False,False,False,R,2019-09-09 07:36:45
3,6410,37.761818,-122.405869,False,0.000,False,False,True,False,False,False,False,False,R,2019-08-06 15:46:25
4,6411,37.732350,-122.414100,False,0.670,False,False,False,False,False,False,False,False,R,2018-10-17 09:54:58


# > **TEST DATA PREPROCESSING**

In [25]:
test_df["Bump"] = test_df["Bump"].astype(int)
test_df["Crossing"] = test_df["Crossing"].astype(int)
test_df["Give_Way"] = test_df["Give_Way"].astype(int)
test_df["Junction"] = test_df["Junction"].astype(int)
test_df["No_Exit"] = test_df["No_Exit"].astype(int)
test_df["Railway"] = test_df["Railway"].astype(int)
test_df["Roundabout"] = test_df["Roundabout"].astype(int)
test_df["Stop"] = test_df["Stop"].astype(int)
test_df["Amenity"] = test_df["Amenity"].astype(int)
test_df.head()

,ID,Lat,Lng,Bump,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Stop,Amenity,Side,timestamp
0,6407,37.786060,-122.390900,0,0.039,0,0,1,0,0,0,0,0,R,2016-04-04 19:20:31
1,6408,37.769609,-122.415057,0,0.202,0,0,0,0,0,0,0,0,R,2020-10-28 11:51:00
2,6409,37.807495,-122.476021,0,0.000,0,0,0,0,0,0,0,0,R,2019-09-09 07:36:45
3,6410,37.761818,-122.405869,0,0.000,0,0,1,0,0,0,0,0,R,2019-08-06 15:46:25
4,6411,37.732350,-122.414100,0,0.670,0,0,0,0,0,0,0,0,R,2018-10-17 09:54:58


In [26]:
f1=list(test_df["timestamp"])                                   #convtring time 
new_time=[]
for i in range (len(f1)):
    new_time.append(f1[i].split()[0])
test_df["Time"]=new_time
test_df

,ID,Lat,Lng,Bump,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Stop,Amenity,Side,timestamp,Time
0,6407,37.786060,-122.390900,0,0.039,0,0,1,0,0,0,0,0,R,2016-04-04 19:20:31,2016-04-04
1,6408,37.769609,-122.415057,0,0.202,0,0,0,0,0,0,0,0,R,2020-10-28 11:51:00,2020-10-28
2,6409,37.807495,-122.476021,0,0.000,0,0,0,0,0,0,0,0,R,2019-09-09 07:36:45,2019-09-09
3,6410,37.761818,-122.405869,0,0.000,0,0,1,0,0,0,0,0,R,2019-08-06 15:46:25,2019-08-06
4,6411,37.732350,-122.414100,0,0.670,0,0,0,0,0,0,0,0,R,2018-10-17 09:54:58,2018-10-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1596,8003,37.812973,-122.362335,0,4.460,0,0,0,0,0,0,0,0,R,2020-06-26 22:32:22,2020-06-26
1597,8004,37.761818,-122.405861,0,0.010,0,0,1,0,0,0,0,0,R,2016-12-03 07:16:30,2016-12-03
1598,8005,37.732260,-122.431970,0,0.431,0,0,1,0,0,0,0,0,R,2017-02-20 06:32:44,2017-02-20
1599,8006,37.786782,-122.390126,0,0.000,1,0,0,0,0,0,0,0,R,2019-10-31 20:35:00,2019-10-31


In [27]:
del test_df["timestamp"]
del test_df['Side']
test_df

,ID,Lat,Lng,Bump,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Stop,Amenity,Time
0,6407,37.786060,-122.390900,0,0.039,0,0,1,0,0,0,0,0,2016-04-04
1,6408,37.769609,-122.415057,0,0.202,0,0,0,0,0,0,0,0,2020-10-28
2,6409,37.807495,-122.476021,0,0.000,0,0,0,0,0,0,0,0,2019-09-09
3,6410,37.761818,-122.405869,0,0.000,0,0,1,0,0,0,0,0,2019-08-06
4,6411,37.732350,-122.414100,0,0.670,0,0,0,0,0,0,0,0,2018-10-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1596,8003,37.812973,-122.362335,0,4.460,0,0,0,0,0,0,0,0,2020-06-26
1597,8004,37.761818,-122.405861,0,0.010,0,0,1,0,0,0,0,0,2016-12-03
1598,8005,37.732260,-122.431970,0,0.431,0,0,1,0,0,0,0,0,2017-02-20
1599,8006,37.786782,-122.390126,0,0.000,1,0,0,0,0,0,0,0,2019-10-31


In [28]:
#test_df=test_df.drop_duplicates(['Time'], keep='last')
#print("The shape of the Weather After removing dublicate is {}.\n\n".format(test_df.shape))
#test_df

In [29]:
X_train.columns

Index(['Lat', 'Lng', 'Distance(mi)', 'Junction', 'Stop'], dtype='object')

In [30]:
X_test = test_df.drop(columns=['ID'])

# You should update/remove the next line once you change the features used for training
X_test = X_test[['Lat', 'Lng', 'Distance(mi)','Junction','Stop']]

y_test_predicted = classifier.predict(X_test)

test_df['Severity'] = y_test_predicted

test_df.head()

,ID,Lat,Lng,Bump,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Stop,Amenity,Time,Severity
0,6407,37.786060,-122.390900,0,0.039,0,0,1,0,0,0,0,0,2016-04-04,2
1,6408,37.769609,-122.415057,0,0.202,0,0,0,0,0,0,0,0,2020-10-28,2
2,6409,37.807495,-122.476021,0,0.000,0,0,0,0,0,0,0,0,2019-09-09,2
3,6410,37.761818,-122.405869,0,0.000,0,0,1,0,0,0,0,0,2019-08-06,2
4,6411,37.732350,-122.414100,0,0.670,0,0,0,0,0,0,0,0,2018-10-17,2


# > **SUBMISSION**

In [31]:
test_df[['ID', 'Severity']].to_csv('/kaggle/working/submission.csv', index=False)